# Descripción del problema 

- $A=(a_{ij})\in M_{d\times n}(\mathbb{Z}^+)$
- $b = \begin{pmatrix} b_1 \\ b_2 \\ \vdots \\  b_n \end{pmatrix},\ \ \  b_i \in \mathbb{Z}^+$ 
- $w = \begin{pmatrix} w_1 & w_2 & \cdots & w_n \end{pmatrix} \in \mathbb{R}^n$. Optimice el gasto $w\cdot u$ sujeto a $A\cdot u = b$

Forma de la solución: 
- $u = \begin{pmatrix} u_1 \\ u_2 \\ \vdots \\  u_n \end{pmatrix},\ \ \  u_i \in \mathbb{Z}^+$

Condiciones: 
- $\displaystyle \sum a_{i\theta} = \alpha \hspace{1cm}$      &&     $\hspace{1cm}\displaystyle \sum b_i = m\alpha$

Lo cual implica:
- $\displaystyle \sum \sum a_{\theta j}u_i = \alpha\sum u_i = \sum{b_i} = m\alpha \hspace{1cm}$ osease $\displaystyle \sum u_i = m$ 

# Operaciones Tropicales
## Definición de las operaciones 

In [1]:
# Busca los indices de elementos repetidos en una lista 
# list: donde voy a buscar 
# elemento: lo que estoy buscando 
def sem(list, elemento): 
    return [indice for indice, x in enumerate(list) if x == elemento]

# OPERACIÓN SUMA - MÍNIMO 
# terms: Elementos a sumar 
# polynom: Me dice si los elementos en terms son polinomios - en cuantas variables 
def plus(terms, polynom = None):
    # Solo son números 
    if polynom == None:
        ans = min(terms)
    else:
        # Miro coeficientes y potencias para agrupar términos semejantes 
        coef = []
        potencias = []
        for i in terms: 
            coef.append(i.coefficients(x)[0][0])
            potencias.append([i.coefficients(x)[0][1]])
        for i in range(polynom - 1): 
            for j in range(len(coef)):
                potencias[j].append(coef[j].coefficients()[0][1])
                coef[j] = coef[j].coefficients()[0][0]
        casians = []
        for i in potencias: 
            coef_surv = sem(potencias, i)
            sublist_coef = [coef[indice] for indice in coef_surv]
            sublist_terms = [terms[indice] for indice in coef_surv]
            if len(coef_surv) != 1: 
                coef_surv = sublist_coef.index(min([coef[indice] for indice in coef_surv]))
                casians.append(sublist_terms[coef_surv])
            else: 
                casians.append(terms[coef_surv[0]])
        ans = []
        for t in casians: 
            if t not in ans: 
                ans.append(t)
    return ans

# OPERACIÓN PRODUCTO - SUMA 
# Definición de la operación Producto
def punto(terms, polynom = None): 
    if polynom == None: 
        ans = 0
        for i in terms: 
            ans += i
    else: 
        coef = []
        for i in terms: 
            coef.append(i.coefficients()[0][0])
        for i in range(polynom - 1): 
            for j in range(len(coef)):
                coef[j] = coef[j].coefficients()[0][0]
        new_coef = [0, 1]
        for i in coef: 
            new_coef[1] *= i
            if i != 1: 
                new_coef[0] += i
        prod = 1 
        for i in terms: 
            prod *= i
        if new_coef[0] != 0: 
            ans = prod / new_coef[1] * new_coef[0]
        else: 
            ans = prod / new_coef[1] 
    return ans  

# Provas 
var('x y z')
plus([3*x**5*y*z**7, 6*x*y**2*z**5, x*y**2*z**5], polynom = 2)
#punto([3*x**5*y*z**7, 6*x*y**2*z**5, x*y**2*z**5], polynom = 3)

[3*x^5*y*z^7, 6*x*y^2*z^5]

In [2]:
# Función busca posibles soluciones 
def find_sum_combinations(target_sum, num_elements, current_combination, all_combinations):
    if num_elements == 0:
        if target_sum == 0:
            all_combinations.append(current_combination)
        return
    
    for value in range(target_sum + 1):
        find_sum_combinations(target_sum - value, num_elements - 1, current_combination + [value], all_combinations)

## Abordaje a la proposición 1.2.2

In [3]:
# FUNCIÓN BUSQUEDA SOLUCIÓN OPTIMA 
# A: Matriz asociada 
# b: Vector independiente 
# w: Vector pesos 
# m: Potencia buscada  
def prop122(A, b, w, m):
    
    # Cálculo de potencia del polinomio p*p*p
    pol = []
    for i in range(w.ncols()):
        pol.append(w[0,i]*x**A[0,i]*y**A[1,i])
    print(f"Polinomio asociado: {pol}")
    producto = []
    contador = 0
    while contador < m-1: 
        if contador == 0: 
            for i in pol: 
                for j in pol: 
                    producto.append(punto([i,j], polynom = 2))
            producto = plus(producto, polynom = 2)
            contador += 1
        else:
            new_producto = []
            for i in pol: 
                for j in producto: 
                    new_producto.append(punto([i,j], polynom = 2))
            producto = plus(new_producto, polynom = 2)
            contador += 1
    print(f"m-ésima potencia: {producto}")

    # Busqueda del coeficiencia optimo
    potencias = []
    coef = []
    polynom = 2
    for i in producto: 
        coef.append(i.coefficients(x)[0][0])
        potencias.append([i.coefficients(x)[0][1]])
    for i in range(polynom - 1): 
        for j in range(len(potencias)):
            potencias[j].append(coef[j].coefficients()[0][1])
            coef[j] = coef[j].coefficients()[0][0]
    ind_ans = potencias.index([b[0][0], b[1][0]])
    cof_opt = coef[ind_ans]
    print(f"Coeficiente optimo: {cof_opt}")

    # Busca en las soluciones w*u = coef optimo && solución del sistema
    all_combinations = [] 
    find_sum_combinations(m, A.ncols(), [], all_combinations)
    ans = []
    contador = 0
    for combination in all_combinations:
        contador += 1
        combination = Matrix(combination).transpose()
        if w*combination == cof_opt and A*combination == b: 
            ans = combination.transpose()
            print(f"Iteraciones: {contador}")
    ## ##
    
    print(f"Solución optima: {ans}")

In [12]:
%time
## Ejemplo del libro 
var('x y z')
A = matrix([[4,3,2,1,0], 
           [0,1,2,3,4]]); 
b = matrix([[5],
           [7]]); 
w = matrix([[2,5,11,7,3]]); 
m = 3; 

prop122(A, b, w, m)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.2 µs
Polinomio asociado: [2*x^4, 5*x^3*y, 11*x^2*y^2, 7*x*y^3, 3*y^4]
m-ésima potencia: [6*x^12, 9*x^11*y, 12*x^10*y^2, 11*x^9*y^3, 7*x^8*y^4, 10*x^7*y^5, 13*x^6*y^6, 12*x^5*y^7, 8*x^4*y^8, 11*x^3*y^9, 17*x^2*y^10, 13*x*y^11, 9*y^12]
Coeficiente optimo: 12
Iteraciones: 22
Solución optima: [1 0 0 1 1]


In [21]:
%time
## Ejemplo de la tésis 
var('x y z')
A = matrix([[3,1,4,1,6], 
            [3,5,2,5,0]]); 
b = matrix([[11],
           [7]]); 
w = matrix([[2,7,1,8,3]]); 
m = 3; 

prop122(A, b, w, m)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs
Polinomio asociado: [2*x^3*y^3, 7*x*y^5, x^4*y^2, 8*x*y^5, 3*x^6]
m-ésima potencia: [6*x^9*y^9, 11*x^7*y^11, 4*x^10*y^8, x^12*y^6, 16*x^5*y^13, 9*x^8*y^10, 2*x^11*y^7, 5*x^13*y^5, 8*x^15*y^3, 21*x^3*y^15, 14*x^6*y^12, 3*x^14*y^4, 6*x^16*y^2, 9*x^18]
Coeficiente optimo: 2
Solución optima: []


## Abordaje desde Programación Lineal Entera

In [6]:
# Solución del Problema con programación ksual
def pli(A, b, w, m): 
    all_combinations = [] 
    find_sum_combinations(m, A.ncols(), [], all_combinations)
    ans = []
    pos_val = []
    contador = 0
    for combination in all_combinations:
        contador += 1
        combination = Matrix(combination).transpose()
        if A*combination == b: 
            ans.append(combination.transpose())
            pos_val.append(w*combination)
    print(f"Soluciones: \n{ans}\nIteraciones: {contador + 1}")
    ind_min = pos_val.index(min(pos_val))
    ans = ans[ind_min]
    print(f"Solución optima: \n{ans}")

In [22]:
%time
## Ejemplo del libro 
var('x y z')
A = matrix([[4,3,2,1,0], 
           [0,1,2,3,4]]); 
b = matrix([[5],
           [7]]); 
w = matrix([[2,5,11,7,3]]); 
m = 3; 

pli(A, b, w, m)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
Soluciones: 
[[0 0 2 1 0], [0 1 0 2 0], [0 1 1 0 1], [1 0 0 1 1]]
Iteraciones: 36
Solución optima: 
[1 0 0 1 1]


In [23]:
%time
## Ejemplo de la tésis 
var('x y z')
A = matrix([[3,1,4,1,6], 
            [3,5,2,5,0]]); 
b = matrix([[11],
           [7]]); 
w = matrix([[2,7,1,8,3]]); 
m = 3; 

pli(A, b, w, m)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs
Soluciones: 
[[0 0 1 1 1], [0 1 1 0 1], [1 0 2 0 0]]
Iteraciones: 36
Solución optima: 
[1 0 2 0 0]


In [9]:
%time
## Ejemplo del libro 
var('x y z')
A = matrix([[4 ,4 , 2, 4, 0,12], 
            [10,11, 8, 6, 4,12],
            [11,10,15,15,21,1]]); 
b = matrix([[105],
            [157],
            [13]]); 
w = matrix([[2,5,11,7,13,1]]); 
m = 13; 

pli(A, b, w, m)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs
Soluciones: 
[]
Iteraciones: 8569


ValueError: min() arg is an empty sequence